In [1]:
# https://leimao.github.io/blog/CSR-Sparse-Matrix-Multiplication/
from __future__ import annotations
from typing import Tuple
import numpy as np


In [1]:
def binsearch(t, key, low = 0, high = len(t) - 1):
    # bisecting the range
    while low < high:
        mid = (low + high)//2
        if t[mid] < key:
            low = mid + 1
        else:
            high = mid
    # at this point 'low' should point at the place
    # where the value of 'key' is possibly stored.
    return low if t[low] == key else -1

NameError: name 't' is not defined

In [ ]:
class CSRMatrix:
    def __init__(self, indptr: np.ndarray, indices: np.ndarray, 
                 data: np.ndarray, shape: Tuple[int, int]) -> None:
        """
        e.g. of CSR representation of a matrix

        A = [[10, 20, 0, 0],
             [0, 0, 0, 12],
             [0, 1, 0, 0]]

        v = [10, 20, 12, 1]
        c = [0, 1, 3, 1]
        r = [0, 2, 3, 4]

        """
        # Row index
        self.indptr = indptr
        # Column index
        self.indices = indices
        self.data = data
        self.shape = shape

        self.dtype = self.data.dtype

    def toarray(self) -> np.ndarray:
        """
        Convert CSR matrix to numpy array.

        Returns:
            np.ndarray: Dense matrix.
        """

        array = np.zeros(self.shape).astype(self.data.dtype)
        num_rows = self.shape[0]
        
        assert num_rows == len(self.indptr)-1

        for i in range(num_rows):
            num_vals = self.indptr[i+1] - self.indptr[i]
            for k in num_vals:
                val = self.data[self.indptr[i]+k]
                j = self.indices[self.indptr[i]+k]
                array[i][j] = val

        return array

    def transpose(self) -> CSRMatrix:
        """
        Transpose of a CSR Matrix.

        e.g. 

        A = [[10, 20, 0, 0],
             [0, 0, 0, 12],
             [0, 1, 0, 0]]

        v = [10, 20, 12, 1]
        c = [0, 1, 3, 1]
        r = [0, 2, 3, 4]

        A^T = [[10, 0, 0],
               [20, 0, 1],
               [0, 0, 0],
               [0, 12, 0]]

        v' = [10, 20, 1, 12]
        c' = [0, 0, 2, 1]
        r' = [0, 1, 3, 3, 4]
        """
        col_2d_idx = self.indices # [0,1,3,1]
        row_2d_idx = np.zeros_like(col_2d_idx) # [0,0,0,0] # row indices of non-zero elements
        k=0
        # self.indptr = [0,2,3,4], self.shape = (3,4)
        num_rows = self.shape[0] # 3
        for i in range(num_rows):
            num_vals = self.indptr[i+1] - self.indptr[i] # 1 # number of non zeros in each rows
            for j in range(num_vals): 
                row_2d_idx[k+j] = i 
            k+=num_vals # row_2d_idx = [0, 0, 1, 2]
        assert k == self.indptr[-1]

        # exchange the row and column index
        new_row_2d_idx = col_2d_idx # [0,1,3,1] 
        new_col_2d_idx = row_2d_idx # [0,0,1,2]

        # Stable sort by using row and column index to find the new nz index of transposed matrix
        ind = np.lexsort((new_col_2d_idx, new_row_2d_idx)) # [0, 1, 3, 2]
        new_row_2d_idx = new_row_2d_idx[ind] # [0, 1, 1, 3] 
        new_col_2d_idx = new_col_2d_idx[ind] # [0, 0, 2, 1]

        # Create CSR matrix
        # O(N)
        indices = new_col_2d_idx # [0, 1, 1, 3]
        # self.data = [10, 20, 12, 1]
        data = self.data[ind] # [10, 20, 1, 12]
        shape = (self.shape[1], self.shape[0]) # (4,3)
        num_rows = shape[0] # 4
        indptr = np.zeros(num_rows + 1).astype(np.int32) # [0, 0, 0, 0, 0]

        for i in new_row_2d_idx:
            indptr[i+1] += 1 # [0, 1, 2, 0, 1] # find the number non zero entries in transposed matrix

        for i in range(num_rows):
            indptr[i+1] += indptr[i] # [0, 1, 3, 3, 4] # take cummulative sum

        indices = np.array(indices).astype(np.int32)
        data = np.array(data).astype(self.dtype)

        csr_matrix = CSRMatrix(indptr=indptr,
                               indices=indices,
                               data=data,
                               shape=shape)

        return csr_matrix

    def append_csr_rows(self, row_mat: CSRMatrix) -> CSRMatrix:
        """
        Append an another CSR Matrix

        e.g. 

        A = [[10, 20, 0, 0],
             [0, 0, 0, 12],
             [0, 1, 0, 0]]

        v = [10, 20, 12, 1]
        c = [0, 1, 3, 1]
        r = [0, 2, 3, 4]

        B = [[0, 0, 0, 1],
             [0, 2, 0, 0]]

        v' = [1,2]
        c' = [3,1]
        r' = [0, 1, 2]

        C = A.append(B) = [[10, 20, 0, 0],
                           [0, 0, 0, 12],
                           [0, 1, 0, 0],
                           [0, 0, 0, 1],
                           [0, 2, 0, 0]]

        v'' = [10, 20, 12, 1, 1, 2]
        c'' = [0, 1, 3, 1, 3, 1]
        r'' = [0, 2, 3, 4, 5, 6]

        Args:
            row_mat (CSRMstrix) : Another CSR matrix with same width self.shape[1] == row_matrix.shape[1]

        Returns:
            CSRMatrix: The new appended matrix
        """
        assert len(row_mat.shape) == 2 # 2D matrix
        assert row_mat.shape[1] == self.shape[1]
        assert self.dtype == row_mat.dtype

        data = np.append(self.data, row_mat.data) # [10, 20, 12, 1, 1, 2]
        indices = np.append(self.indices, row_mat.indices) # append to column indices
        shape = (self.shape[0] + row_mat.shape[0], self.shape[1]) # shape of the appended matrix

        indptr = row_mat.indptr.copy() + self.indptr[-1] # [0, 1, 2] + 4 = [4, 5, 6]
        indptr = np.append(self.indptr[:-1], indptr) # [0, 2, 3, 4, 5, 6]

        csr_matrix = CSRMatrix(indptr=indptr,
                               indices=indices,
                               data=data,
                               shape=shape)
        return csr_matrix

    def add_csr_mat(self, b:CSRMatrix) -> CSRMatrix:
        """
        Add two csr matrices together

        A = [[10, 20, 0],
             [0, 0, 0],
             [0, 1, 0]]

        v = [10, 20, 1]
        c = [0, 1, 1]
        r = [0, 2, 2, 3]

        B = [[0 ,0 ,1],
             [0, 1, 0], 
             [2, 3, 0]]
        
        v' = [1, 1, 2, 3]
        c' = [2, 1, 0, 1]
        r' = [0, 1, 2, 4]

        C = A + B = [[10, 20, 1],
                     [0, 1, 0],
                     [2, 4, 0]]

        v'' = [10, 20, 1, 1, 2, 4]
        c'' = [0, 1, 2, 1, 0, 1]
        r'' = [0, 3, 4, 6] 

        Args:
            b: Another CSR matrix with same shape
        
        Returns:
            CSRMatirx: sum of the two matrices
        """

        assert b.shape == self.shape, "Size of two matrices should be same"

        col_2d_idx_1 = self.indices # [0,1,1]
        col_2d_idx_2 = b.indices # [2,1,0,1]

        row_2d_idx_1 = np.zeros_like(col_2d_idx_1)
        row_2d_idx_2 = np.zeros_like(col_2d_idx_2)
        k_1 = 0
        k_2 = 0
        num_rows = self.shape[0] # 3
        for i in range(num_rows):
            num_vals_1 = self.indptr[i+1] - self.indptr[i] # 1 # number of non zeros in each row 
            num_vals_2 = b.indptr[i+1] - b.indptr[i]
            for j in range(num_vals_1): 
                row_2d_idx_1[k_1+j] = i 
            for j in range(num_vals_2):
                row_2d_idx_2[k_2+j] = i
            k_1 += num_vals_1 # row_2d_idx = [0, 0, 2]
            k_2 += num_vals_2 # row_2d_idx_1 = [0,1,2,2]
        assert k_1 == self.indptr[-1]
        assert k_2 == b.indptr[-1]

        """
        [10,20,1]
        [0,0,2] [0,1,1]
        [1,1,2,3]
        [0,1,2,2] [2,1,0,1]

        [10, 20, 1] [(0,0), (0,1), (2,1)]
        [1, 1, 2, 3] [(0,2), (1,1), (2,0), (2,1)]
        """


        
        mat_idx_1 = tuple(zip(row_2d_idx_1, col_2d_idx_1))
        mat_idx_2 = tuple(zip(row_2d_idx_2, col_2d_idx_2))

        apos , bpos = 0, 0
        data = []
        col_idx = []
        row_idx = []
        while ((apos < len(self.data)) and (bpos < len(b.data))):
            if (row_2d_idx_1[apos] > row_2d_idx_2[bpos]) or (col_2d_idx_1[apos] > col_2d_idx_2[bpos]):
                data.append(b.data[bpos])
                col_idx.append(col_2d_idx_2[bpos])
                row_idx.append(row_2d_idx_1[bpos])

                bpos+=1

            elif (row_2d_idx_1[apos] < row_2d_idx_2[bpos]) or (col_2d_idx_1[apos] < col_2d_idx_2[bpos]):
                data.append(b.data[apos])
                col_idx.append(col_2d_idx_1[apos])
                row_idx.append(row_2d_idx_1[apos])

                apos+=1

            else:
                addedval = self.data[apos] + b.data[bpos]

                if (addedval != 0):
                    data.append[addedval]
                    col_idx.append(col_2d_idx_2[bpos])
                    row_idx.append(row_2d_idx_1[bpos])
                
                apos +=1 
                bpos +=1
        
        while (apos < len(self.data)):
            data.append(b.data[apos])
            col_idx.append(col_2d_idx_1[apos])
            row_idx.append(row_2d_idx_1[apos])
            apos+=1

        while (bpos < len(b.data)):
            data.append(b.data[bpos])
            col_idx.append(col_2d_idx_2[bpos])
            row_idx.append(row_2d_idx_1[bpos])
            bpos+=1

        indices = col_idx
        shape = self.shape
        num_rows = shape[0]
        indptr = np.zeros(num_rows + 1).astype(np.int32) # [0, 0, 0, 0, 0]

        for i in row_idx:
            indptr[i+1] += 1 # [0, 1, 2, 0, 1] # find the number non zero entries in transposed matrix

        for i in range(num_rows):
            indptr[i+1] += indptr[i] # [0, 1, 3, 3, 4] # take cummulative sum

        indices = np.array(indices).astype(np.int32)
        data = np.array(data).astype(self.dtype)

        csr_matrix = CSRMatrix(indptr=indptr,
                               indices=indices,
                               data=data,
                               shape=shape)

        return csr_matrix